# Project 1: LHC anomaly detection challenge

**Dataset:** https://uwmadison.box.com/s/702f3xsndlbpc18ud54vemewku63uypg

**Grading:**
These projects will be peer-graded using the rubric listed below. 

**Deadline:** 
The project will be due by class time on Monday, October 6th at 4:00pm Central Time. Please upload a `zip` file to Canvas of the following: 
- Your completed notebook (`.ipynb` format)
- Your completed notebook (`.html` format)
- A PDF of an extended abstract, written in LaTeX and no more than 2 pages in length, introducing the problem, summarizing your methods and work, reflecting on your results, listing what future work you would explore, and showing at least one figure. You may use any LaTeX format you like, including 2-column layouts if you prefer. 

## Project description
In this project, you will analyze a dataset consisting of 1 million simulated LHC dijet events for which some number of events correspond to a new, Beyond-the-Standard-Model particle. Below you will find some basic imports and plotting style code as well as instructions on how to load the dataset. Your job is to design an analysis that is able to detect this resonant anomaly in a data-driven manner using weakly-supervised anomaly detection. 

## Additional resources
- https://journals.aps.org/prd/pdf/10.1103/PhysRevD.99.014038
- https://arxiv.org/pdf/2101.08320

## Imports & utility function definitions

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
import seaborn as sns
import random
hep.style.use("CMS")

### ML-related imports
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, TensorDataset, DataLoader
from livelossplot import PlotLosses
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc

### plotting_setup 
%config InlineBackend.figure_format = "retina"

# plt.rcParams["figure.figsize"] = (9,6)
plt.rcParams["figure.figsize"] = (7,5)

medium_size = 12
large_size = 15

plt.rc("font", size=medium_size)          # default text sizes
plt.rc("xtick", labelsize=medium_size)    # xtick labels
plt.rc("ytick", labelsize=medium_size)    # ytick labels
plt.rc("legend", fontsize=medium_size)    # legend
plt.rc("axes", titlesize=large_size)      # axes title
plt.rc("axes", labelsize=large_size)      # x and y labels
plt.rc("figure", titlesize=large_size)    # figure title

SEED = 1234 
def set_deterministic(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) 

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False
SEED = 2001 # for reproducibility 
set_deterministic(seed=SEED)

In [ ]:
def get_mjj(df): 
    E1 = np.sqrt(df.pxj1**2 + df.pyj1**2 + df.pzj1**2 + df.mj1**2)
    E2 = np.sqrt(df.pxj2**2 + df.pyj2**2 + df.pzj2**2 + df.mj2**2)
    
    Ex = df.pxj1 + df.pxj2
    Ey = df.pyj1 + df.pyj2
    Ez = df.pzj1 + df.pzj2
    E  = E1 + E2
    return np.sqrt(np.maximum(E**2 - (Ex**2 + Ey**2 + Ez**2), 0.0))

In [ ]:
def make_regions(m0s, sigregion_width, bkgregion_width):
    """
    Generates a list of dictionaries with the signal and background regions for each mass hypothesis
    Also checks that the signal regions overlap
    """
    regions = []
    all_intervals = []
    for m0 in m0s:
        sig_low = m0 - sigregion_width / 2
        sig_high = m0 + sigregion_width / 2
        bkg_left_low = sig_low - bkgregion_width
        bkg_left_high = sig_low
        bkg_right_low = sig_high
        bkg_right_high = sig_high + bkgregion_width

        regions.append(
            {
                "m0": m0,
                "sigregion": (sig_low, sig_high),
                "bkgregion_left": (bkg_left_low, bkg_left_high),
                "bkgregion_right": (bkg_right_low, bkg_right_high)
            }
        )

        # Collect all intervals
        all_intervals.extend(
            [
                (bkg_left_low, bkg_left_high),
                (sig_low, sig_high),
                (bkg_right_low, bkg_right_high)
            ]
        )

    if not(regions[0]["m0"] >= regions[1]["m0"] - sigregion_width):
        raise ValueError("There are some non-overlapping signal regions!")
    return regions

In [ ]:
def dataset_generator(df, input_vars, region, batch_size):
    """
    Generates train, val, test dataloaders and datasets for a given signal and sideband region
    """
    in_any_reg = np.array(df["m_jj"].between(region["bkgregion_left"][0], region["bkgregion_right"][1]))

    # Creating separate df and adding label column to new ds
    df_in_reg = df[in_any_reg].copy()
    df_in_reg["labels"] = df_in_reg["m_jj"].between(region["sigregion"][0], region["sigregion"][1]).astype(int)

    # I do it this way to keep track of ordering of events.
    # That way I can later plot kinematic distributions of events classified as signal
    df_train, df_valtest = train_test_split(df_in_reg, test_size=0.2, random_state=SEED)
    df_val, df_test = train_test_split(df_valtest, test_size=0.5, random_state=SEED)

    X_train = df_train.filter(items=input_vars).to_numpy(dtype=np.float32)
    scaler = StandardScaler().fit(X_train)

    X_train = torch.tensor(scaler.transform(X_train))
    y_train = torch.tensor(np.array(df_train["labels"], dtype=np.float32).reshape(-1,1))

    # Other ds scaled separately
    X_val = torch.tensor(scaler.transform(df_val.filter(items=input_vars).to_numpy(dtype=np.float32)))
    y_val = torch.tensor(np.array(df_val["labels"], dtype=np.float32).reshape(-1,1))

    X_test = torch.tensor(scaler.transform(df_test.filter(items=input_vars).to_numpy(dtype=np.float32)))
    y_test = torch.tensor(np.array(df_test["labels"], dtype=np.float32).reshape(-1,1))   
    
    train_ds = TensorDataset(X_train, y_train)
    val_ds = TensorDataset(X_val, y_val)
    test_ds = TensorDataset(X_test, y_test)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader, (dict(train=df_train, val=df_val, test=df_test)), (dict(train=train_ds, val=val_ds, test=test_ds))

In [ ]:
def get_uncorr_vars(corr_df, target_var, threshold, plot=False):
    """
    Identifies variables that are uncorrelated with the target variable. Also can plot the correlations.
    """
    mjj_corr = corr_df[target_var].drop(target_var)
    least_correlated = mjj_corr.abs().sort_values()
    if plot:
        fig, ax = plt.subplots(figsize=(10, 5))
        ax.bar(least_correlated.index, least_correlated.values)

    return list(least_correlated[least_correlated < threshold].index), (fig, ax) if plot else None

In [ ]:
def plot_logs(logs, region=None):
    """
    Plots the training and validation loss curves.
    """
    fig, ax = plt.subplots(1, 1, figsize=(7,5))
    ax.plot(logs["loss"], label="Training Loss")
    ax.plot(logs["val_loss"], label="Validation Loss")
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Loss")
    ax.legend()
    title = "Training and Validation Loss"
    if region is not None:
        title += f" ($m_0$ = {region['m0']} GeV)"
    ax.set_title(title)
    return fig, ax

In [ ]:
def plot_roc_curve(dfs, dss, model):
    """
    Plots the ROC curve for the given model and dataset.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    with torch.no_grad():
        X_test_tensor, y_test_tensor = dss["test"].tensors
        y_test_preds = model(X_test_tensor.to(device))

    fpr, tpr, thresholds = roc_curve(dfs["test"]["labels"], y_test_preds.cpu().numpy())
    roc_auc = auc(fpr, tpr)
    fig, ax = plt.subplots()
    ax.plot(fpr, tpr, color="darkorange", lw=2, label="ROC curve (area = %0.2f)" % roc_auc)
    ax.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel("False Positive Rate")
    ax.set_ylabel("True Positive Rate")
    ax.set_title("Receiver Operating Characteristic")
    ax.legend(loc="lower right")
    ax.grid()
    return fig, ax

In [ ]:
def plot_rslts_grids(rslts):
    n_cols = 3
    n_rows = int(np.ceil(len(rslts) / n_cols))
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 5 * n_rows), tight_layout=True)
    axes = axes.flatten()
    for i, rslt in enumerate(rslts):
        fpr, tpr, thresholds = rslt["roc"]
        ax = axes[i]
        ax.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % rslt["auc"])
        ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        ax.set_xlim([0.0, 1.0])
        ax.set_ylim([0.0, 1.05])
        ax.set_xlabel('False Positive Rate')
        ax.set_ylabel('True Positive Rate')
        ax.set_title(f"ROC for $m_0$ = {rslt['region']['m0']} GeV")
        ax.legend(loc="lower right")
        ax.grid()
    plt.show()

    # Make grid of loss plots from rslts. With 3 columns
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 5 * n_rows), tight_layout=True)
    axes = axes.flatten()
    for i, rslt in enumerate(rslts):
        ax = axes[i]
        ax.plot(rslt["logs"]["loss"], label="Training Loss")
        ax.plot(rslt["logs"]["val_loss"], label="Validation Loss")
        ax.set_xlabel("Epoch")
        ax.set_ylabel("Loss")
        ax.legend()
        ax.set_title(f"Training and Validation Loss ($m_0$ = {rslt['region']['m0']} GeV)")
        ax.grid()
    plt.show()

    # Plot model output scores for all mass points on the same plot
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 5 * n_rows), tight_layout=True)
    axes = axes.flatten()
    for i, rslt in enumerate(rslts):
        ax = axes[i]
        y_test_preds = rslt["model"](rslt["X_test_tensor"].to(device)).detach().cpu().numpy()
        ax.hist(y_test_preds, bins=100, range=(0, 1))
        ax.set_xlabel("Model Output Score")
        ax.set_ylabel("Counts")
        ax.set_title(f"Model Output Scores on Test Set ($m_0$ = {rslt['region']['m0']} GeV)")
    plt.show()

    # Plot AUC vs mass hypothesis
    m0s = [rslt["region"]["m0"] for rslt in rslts]
    aucs = [rslt["auc"] for rslt in rslts]
    plt.plot(m0s, aucs, marker='o')
    plt.xlabel(r"$m_0$ [GeV]")
    plt.ylabel("AUC")
    plt.title("AUC vs Mass Hypothesis")
    plt.grid()
    plt.show()

## Data exploration

The dataset consists of 14 variables, for the highest-momentum jet (j1) and the second-highest momentum jet (j2): 
- The jet mass: $m_j$
- The 3-momenta: $p_x$, $p_y$, $p_z$
- 3 variables characterizing the jet substructure ("$n$-subjettiness"): $\tau_1$, $\tau_2$, and $\tau_3$

To the dataframe that holds these variables, we add the invariant mass of the dijets.

In [ ]:
df = pd.read_hdf("./data/lhc_mystery_anomaly_data.h5")
df["m_jj"] = get_mjj(df)
df.head()

Considering the cylindrical geometry of the ATLAS & CMS experiments at the LHC, we convert the kinematic quantities to more appropriate coordinates.

$$
    \eta = \text{arctanh}\left(\frac{\vec|\vec p| + p_z}{|\vec p| - p_z}\right)
$$

$$
    \phi = \text{arctan2}\left(\frac{p_y}{px}\right)
$$

$$
    p_T = \sqrt{p_x^2 + p_y^2}
$$

In [ ]:
df["ptj1"] = np.sqrt(df["pxj1"]**2 + df["pxj1"]**2)
df["ptj2"] = np.sqrt(df["pxj2"]**2 + df["pxj2"]**2)

df["phij1"] = np.arctan2(df["pyj1"], df["pxj1"])
df["phij2"] = np.arctan2(df["pyj2"], df["pxj2"])

df["etaj1"] = np.arcsinh(df["pzj1"]/df["ptj1"])
df["etaj2"] = np.arcsinh(df["pzj2"]/df["ptj2"])

# get only columns for jet 1
df.filter(like="j1", axis=1).head()

We will also compute the ratio of the n-subjetiness variables.

In [ ]:
epsilon = 1e-8

df["tau21j1"] = df["tau2j1"]/df["tau1j1"].replace(0, epsilon)
df["tau32j1"] = df["tau3j1"]/df["tau2j1"].replace(0, epsilon)

df["tau21j2"] = df["tau2j2"]/df["tau1j2"].replace(0, epsilon)
df["tau32j2"] = df["tau3j2"]/df["tau2j2"].replace(0, epsilon)

We now plot the given and computed features, as well as the invariant mass, in order to gain some insight into how they are distributed.

In [ ]:
plt.hist(df["m_jj"], bins=100, range=(1500, 6500), histtype="step", lw=1)
plt.xlabel(r"$m_{jj}$")
plt.ylabel("Counts")
plt.title("Dijet Mass")
plt.show()

We now visualize the features.

In [ ]:
plt.hist(df["mj1"], bins=100, range=(0, 1000), histtype="step", label="Jet 1", density=True)
plt.hist(df["mj2"], bins=100, range=(0, 1000), histtype="step", label="Jet 2", density=True)
plt.title("Jet Mass Distribution")
plt.xlabel("$m_J$ [GeV]")
plt.ylabel("A.U.")
plt.legend()
plt.show()

In [ ]:
plt.hist(df.phij1, bins=30, range=(-np.pi, np.pi), histtype="step", label="Jet 1", density=True)
plt.hist(df.phij2, bins=30, range=(-np.pi, np.pi), histtype="step", label="Jet 2", density=True)
plt.title("$\phi$ Distribution of Jets")
plt.xlabel("$\phi$")
plt.ylabel("A.U.")
plt.legend()
plt.show()

In [ ]:
plt.hist(df.etaj1, bins=100, range=(-3,3), histtype="step", label="Jet 1", density=True)
plt.hist(df.etaj2, bins=100, range=(-3, 3), histtype="step", label="Jet 2", density=True)
plt.title("$\eta$ Distribution of Jets")
plt.xlabel("$\eta$")
plt.ylabel("A.U.")
plt.legend()
plt.show()

In [ ]:
plt.hist(df.ptj1, bins=100, range=(-100, 3500), histtype="step", label="Jet 1", density=True)
plt.hist(df.ptj2, bins=100, range=(-100, 3500), histtype="step", label="Jet 2", density=True)
plt.title("$p_T$ Distribution of Jets")
plt.xlabel("$p_T$ [GeV]")
plt.ylabel("A.U.")
plt.legend()
plt.show()

In [ ]:
for i in range(1, 4):
    plt.hist(df[f"tau{i}j1"], bins=100, range=(0, 1500), histtype="step", label="Jet 1", density=True)
    plt.hist(df[f"tau{i}j2"], bins=100, range=(0, 1500), histtype="step", label="Jet 2", density=True)
    plt.title(f"$\\tau_{i}$ Distribution of Jets")
    plt.xlabel(f"$\\tau_{i}$")
    plt.ylabel("A.U.")
    plt.yscale("log")
    plt.legend()
    plt.show()

In [ ]:
plt.hist(df["tau21j1"], bins=100, range=(0, 1), histtype="step", label="Jet 1", density=True)
plt.hist(df["tau21j2"], bins=100, range=(0, 1), histtype="step", label="Jet 2", density=True)
plt.title(r"$\tau$21 Distribution of Jets")
plt.xlabel(r"$\tau$21")
plt.ylabel("A.U.")
plt.legend()
plt.show()

plt.hist(df["tau32j1"], bins=100, range=(0, 1), histtype="step", label="Jet 1", density=True)
plt.hist(df["tau32j2"], bins=100, range=(0, 1), histtype="step", label="Jet 2", density=True)
plt.title(r"$\tau$32 Distribution of Jets")
plt.xlabel(r"$\tau$32")
plt.ylabel("A.U.")
plt.legend()
plt.show()

## Input correlation evaluation

With the method we will be using, its important that the input features are not significantly correlated with the di-jet mass. Thus, we look at the Pearson correlation between each feature and $m_{jj}$, on which we will apply a cut later on to exclude those features which are too highly correlated.

In [ ]:
df = df.filter(regex="^(?!.*(px|py|pz)).*$") # Remove px, py, pz columns since we will use eta phi and pt
cols = ["m_jj"] + [col for col in df.columns if "j1" in col] + [col for col in df.columns if "j2" in col]

# We filter out x, y and z variables since we will use eta phi and pt
corr_df = df[cols].corr()
fig, ax = plt.subplots(figsize=(14, 12))

# Set vmin and vmax to center the colormap around 0
sns.heatmap(corr_df, cmap="coolwarm", vmin=-1, vmax=1, fmt=".2f", linewidths=0.5, annot=True, annot_kws={"size": 10}, ax=ax)
ax.tick_params(axis="both", which="both", length=0)
# Highlight m_jj row and column
for i, label in enumerate(corr_df.columns):
    if label == "m_jj":
        ax.add_patch(plt.Rectangle((i, 0), 1, len(corr_df), fill=False, edgecolor="black", lw=2))
        ax.add_patch(plt.Rectangle((0, i), len(corr_df), 1, fill=False, edgecolor="black", lw=2))
        break
plt.title("Feature Correlation Matrix")
plt.savefig("corr_mtrx.pdf")
plt.show()

The threshold we will be using will he of 0.10. Note that tighter cuts were tested during development, but they made no appreciable difference on the results.

In [ ]:
CORR_THRESHOLDS = 0.10
input_vars, (fig, ax) = get_uncorr_vars(df.corr(), "m_jj", CORR_THRESHOLDS, plot=True)
ax.tick_params(axis="x", rotation=45)
ax.axhline(CORR_THRESHOLDS, color="red", linestyle="--", label=f"Threshold = {CORR_THRESHOLDS}")
ax.grid(axis="y")
ax.legend()
ax.set_ylabel("Correlation with $m_{jj}$")
ax.set_title("Variables Ranked by Correlation with $m_{jj}$")
# save to pdf
plt.savefig("corr_bars.pdf")
plt.show()
input_vars

## Setting up weakly-supervised analysis & Preprocessing data

We first define the signal and sideband regions using the function `make_regions`. Its important that the signal regions have a full coverage of the di-jet mass distribution, so we also plot the mass hypotheses as well as the border of each signal region to visually ensure that that is the case.

In [ ]:
bkgregion_width = 100
sigregion_width = 200
m0_interval = 100
m0_min = 2850
m0_max = 5000
m0s = np.arange(m0_min, m0_max + 1, m0_interval)

regions = make_regions(m0s, sigregion_width, bkgregion_width)

# Plot all m0 as vertical lines
for region in regions:
    plt.axvline(region["m0"], color="gray", linestyle="--", alpha=0.5)
    plt.axvline(region["sigregion"][1], color="red", linestyle="-", alpha=0.7)
    plt.axvline(region["sigregion"][0], color="blue", linestyle="-", alpha=0.7)
    plt.text(region["m0"], 50, f"{region['m0']}", rotation=90, verticalalignment='bottom', horizontalalignment='right', fontsize=8)
    plt.axvspan(region["bkgregion_left"][0], region["bkgregion_left"][1], color='orange', alpha=0.3)
    plt.axvspan(region["sigregion"][0], region["sigregion"][1], color='blue', alpha=0.3)
    plt.axvspan(region["bkgregion_right"][0], region["bkgregion_right"][1], color='orange', alpha=0.3)
plt.hist(df["m_jj"], bins=100, range=(2500, 6800))
plt.xlabel(r"$m_{jj}$")
plt.ylabel("Counts")
plt.title("Dijet Mass with Signal and Background Regions")
plt.show()

print("Mass hypotheses (m0s) used (GeV):")
print(m0s)

We start with just two features, namely `tau3j1` and `tau2j1`, and then move on to use a correlation threshold of 0.1 to include more features that are not significantly correlated with the di-jet invariant mass later on.

In [ ]:
input_vars = ["tau3j1", "tau2j1"]

We choose an arbitrary region to run the training workflow. For this particular region, we construct the training, validation and testing datasets using the utility function `dataset_generator`. This function takes only events from the di-jet mass window to construct these datasets. It also takes care of assigning the proper labels: 0 to di-jets in the sidebands, and 1 to di-jets in the hypothesis signal region.

In [ ]:
BATCH_SIZE = 256
region = regions[1]

# Data preprocessing happens in dataset_generator
train_loader, val_loader, test_loader, dfs, dss = dataset_generator(df, input_vars, region, batch_size=BATCH_SIZE)
print(f"Train size: {len(dfs['train'])}, Val size: {len(dfs['val'])}, Test size: {len(dfs['test'])}")

## Defining model & training

We define a simple MLP class that uses `ReLU` activation function, as well as optional dropout, and outputs a single value which represents the predicted probability of an input being in the signal region. Additionally, we define a training function which uses a binary cross-entropy loss with the Adam optimizer.

In [ ]:
HIDDEN_DIM = 32
HIDDEN_LAYERS = 3
DROP_RATE = 0

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=HIDDEN_DIM, hidden_layers = HIDDEN_LAYERS, dropout_rate=DROP_RATE):
        super().__init__()
        layers = nn.ModuleList(
            [
                nn.Linear(input_dim, hidden_dim), 
                nn.ReLU(), 
                nn.Dropout(dropout_rate)
            ]
        )
        for _ in range(hidden_layers - 1):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
        layers.append(nn.Linear(hidden_dim, 1))
        layers.append(nn.Sigmoid()) # output is 1-dimensional for binary classification
        layers = nn.ModuleList(layers)
        self.MLP = nn.Sequential(*layers)
    def forward(self, x):
        return self.MLP(x)
    def count_trainable_params(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad) 

In [ ]:
N_EPOCHS = 20
LR = 0.01

def train(model, train_loader, val_loader, n_epochs=N_EPOCHS, lr=LR):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    ### Initialize the model, optimizer, and loss function -- what should the input dimensionality be?
    model = model.to(device) # move onto the GPU, if present
    # loss_fn = nn.BCEWithLogitsLoss() # BCE = Binary Cross Entropy
    loss_fn = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    liveloss = PlotLosses(figsize=(9, 4)) 
    logs = {"loss": [], "val_loss": []}

    ### Training loop
    for epoch in range(n_epochs):

        ### train
        # Set the model to training mode
        model.train()
        total_train_loss = 0
        # Iterate over the training DataLoader
        for x_batch, y_batch in train_loader:
            # Zero the gradients because by default they accumulate because
            optimizer.zero_grad()
            
            ### Move the batch onto the GPU, if present
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)

            # Forward pass
            out = model(x_batch)
            # Compute loss
            loss = loss_fn(out, y_batch)
            # Backward pass
            loss.backward()
            # Update the weights after the gradients have been computed for this batch
            optimizer.step()
            # Accumulate the training loss
            total_train_loss += loss.item()

        # Average training loss over all batches in this epoch
        train_loss_per_batch = total_train_loss / len(train_loader)
        logs['loss'].append(train_loss_per_batch)

        ### validate
        # Set the model to evaluation mode
        model.eval()
        total_val_loss = 0
        # Iterate over the validation DataLoader
        for x_batch, y_batch in val_loader:
            
            ### Move the batch onto the GPU, if present
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)

            # Forward pass
            out = model(x_batch)
            # Compute loss
            loss = loss_fn(out, y_batch)
            # Accumulate the validation loss
            total_val_loss += loss.item()

        # Average validation loss over all validation batches
        val_loss_per_batch = total_val_loss / len(val_loader)
        # Log the validation loss
        logs['val_loss'].append(val_loss_per_batch)

        # Update the liveloss plot
        liveloss.update(
            {
                "loss": logs["loss"][-1],
                "val_loss": logs["val_loss"][-1]
            }
        )
        liveloss.send()
        print(f'Epoch {epoch}, Loss: {loss.item()}')

    return model, logs

We now run the training of the model (with 3 hidden layers, each with dimension 32) for 20 epochs and a learning rate of 0.01 on the previously chosen mass window.

In [ ]:
set_deterministic(seed=SEED)
model = MLP(input_dim=len(input_vars), hidden_dim=HIDDEN_DIM, hidden_layers=HIDDEN_LAYERS)
print("Number of trainable parameters:", model.count_trainable_params())

In [ ]:
model, logs = train(model, train_loader=train_loader, val_loader=val_loader)

A look at the loss curves shows that the model did not seem to be able to effectively minimize the loss and thus was not able to learn to differentiate di-jets from the signal region from those from the side-bands.

In [ ]:
fig, ax = plot_logs(logs, region)
ax.grid()
plt.show()

## Evaluating the model

From the loss curves seen before above, its clear that the model was not able to effectively learn how to discriminate between di-jets in the signal region and those in the side-bands. We evaluate the trained model on the testing dataset in order to confirm this.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
with torch.no_grad():
    X_test_tensor, y_val_tensor = dss["test"].tensors
    y_test_preds = model(X_test_tensor.to(device))

If we plot the output distribution, we can see that the output score of all di-jets is ~0.5.

In [ ]:
plt.hist(y_test_preds.cpu().numpy(), bins=100)
plt.xlabel("Model Output Score")
plt.ylabel("Counts")
plt.title("Model Output Scores on Test Set")
plt.show()

Moreover, if we plot the ROC curve, its clear that the model performs no better than random choice.

In [ ]:
# Making ROC plot
fig, ax = plot_roc_curve(dfs, dss, model)
plt.show()

### Scan

With the workflow established, we can now run the training and evaluation for all mass hypotheses and collect the results.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
rslts = []

for region in regions:
    print(region)
    set_deterministic(seed=SEED)

    train_loader, val_loader, test_loader, dfs, dss = dataset_generator(df, input_vars, region, batch_size=BATCH_SIZE)

    model = MLP(input_dim=len(input_vars), hidden_dim=HIDDEN_DIM, hidden_layers=HIDDEN_LAYERS)

    if len(dfs["train"]) / model.count_trainable_params() <= 10:
        print("Not enough training data for the number of parameters in the model.")
        print(f"Train size: {len(dfs['train'])}, Val size: {len(dfs['val'])}, Test size: {len(dfs['test'])}")
        break

    model, logs = train(model, n_epochs=N_EPOCHS, lr=LR, train_loader=train_loader, val_loader=val_loader)
    
    with torch.no_grad():
        X_test_tensor, y_test_tensor = dss["test"].tensors
        y_test_preds = model(X_test_tensor.to(device)).cpu().numpy()
    fpr, tpr, thresholds = roc_curve(y_test_tensor, y_test_preds)

    rslts.append(
        {
            "region": region,
            "model": model,
            "logs": logs,
            "roc": (fpr, tpr, thresholds),
            "auc": auc(fpr, tpr),
            "X_test_tensor": X_test_tensor,
            "y_test_tensor": y_test_tensor,
        }
    )

In [ ]:
plot_rslts_grids(rslts)

These results show no evidence of the model having been able to discriminate between signal region and side-band region di-jets better than random choice, and thus we have not found the signal we are searching for yet.

## Including more features

We now include only those features that pass a correlation cut of 0.1.

In [ ]:
CORR_THRESHOLDS = 0.1
input_vars, _ = get_uncorr_vars(df.corr(), "m_jj", CORR_THRESHOLDS, plot=False)
input_vars

### Scan

Now that the workflow is setup, we perform a scan over the entire di-jet mass range. We use the same di-jet mass regions defined previously.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
rslts = []

for region in regions:
    print(region)
    set_deterministic(seed=SEED)

    train_loader, val_loader, test_loader, dfs, dss = dataset_generator(df, input_vars, region, batch_size=BATCH_SIZE)

    model = MLP(input_dim=len(input_vars), hidden_dim=HIDDEN_DIM, hidden_layers=HIDDEN_LAYERS)

    if len(dfs["train"]) / model.count_trainable_params() <= 10:
        print("Not enough training data for the number of parameters in the model.")
        print(f"Train size: {len(dfs['train'])}, Val size: {len(dfs['val'])}, Test size: {len(dfs['test'])}")
        break

    model, logs = train(model, n_epochs=N_EPOCHS, lr=LR, train_loader=train_loader, val_loader=val_loader)
    
    with torch.no_grad():
        X_test_tensor, y_test_tensor = dss["test"].tensors
        y_test_preds = model(X_test_tensor.to(device)).cpu().numpy()
    fpr, tpr, thresholds = roc_curve(y_test_tensor, y_test_preds)

    rslts.append(
        {
            "region": region,
            "model": model,
            "logs": logs,
            "roc": (fpr, tpr, thresholds),
            "auc": auc(fpr, tpr),
            "X_test_tensor": X_test_tensor,
            "y_test_tensor": y_test_tensor,
        }
    )


In [ ]:
plot_rslts_grids(rslts)

## Additional scans with other hyperparameter values

The above results clearly show that the classifier mostly performed around the same as random chance with very little deviation from this. We now do an additional scan over the same regions, but with a larger model. We do this by having 4 hidden layers instead of 3. Note that this limits the reach of the search, as the current implementation only goes up to the mass hypothesis window for which there are enough training events given a certain number of trainable parameters.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
rslts = []

for region in regions:
    print(region)
    set_deterministic(seed=SEED)

    train_loader, val_loader, test_loader, dfs, dss = dataset_generator(df, input_vars, region, batch_size=BATCH_SIZE)

    model = MLP(input_dim=len(input_vars), hidden_dim=HIDDEN_DIM, hidden_layers=4)

    if len(dfs["train"]) / model.count_trainable_params() <= 10:
        print("Not enough training data for the number of parameters in the model.")
        print(f"Train size: {len(dfs['train'])}, Val size: {len(dfs['val'])}, Test size: {len(dfs['test'])}")
        break

    model, logs = train(model, n_epochs=N_EPOCHS, lr=LR, train_loader=train_loader, val_loader=val_loader)
    
    with torch.no_grad():
        X_test_tensor, y_test_tensor = dss["test"].tensors
        y_test_preds = model(X_test_tensor.to(device)).cpu().numpy()
    fpr, tpr, thresholds = roc_curve(y_test_tensor, y_test_preds)

    rslts.append(
        {
            "region": region,
            "model": model,
            "logs": logs,
            "roc": (fpr, tpr, thresholds),
            "auc": auc(fpr, tpr),
            "X_test_tensor": X_test_tensor,
            "y_test_tensor": y_test_tensor,
        }
    )


In [ ]:
plot_rslts_grids(rslts)

Unfortunately, the previously observed null results still hold for this new and slightly larger model. Adding more model trainable parameters by using a larger model would limit the range of the di-jet mass range we could scan, unless a different binning strategy was adopted for the region definition.

### Smaller model

We now try a smaller model, which might help with any overfitting that might be happening.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
rslts = []

for region in regions:
    print(region)
    set_deterministic(seed=SEED)

    train_loader, val_loader, test_loader, dfs, dss = dataset_generator(df, input_vars, region, batch_size=BATCH_SIZE)

    model = MLP(input_dim=len(input_vars), hidden_dim=16, hidden_layers=HIDDEN_DIM)

    if len(dfs["train"]) / model.count_trainable_params() <= 10:
        print("Not enough training data for the number of parameters in the model.")
        print(f"Train size: {len(dfs['train'])}, Val size: {len(dfs['val'])}, Test size: {len(dfs['test'])}")
        break

    model, logs = train(model, n_epochs=N_EPOCHS, lr=LR, train_loader=train_loader, val_loader=val_loader)
    
    with torch.no_grad():
        X_test_tensor, y_test_tensor = dss["test"].tensors
        y_test_preds = model(X_test_tensor.to(device)).cpu().numpy()
    fpr, tpr, thresholds = roc_curve(y_test_tensor, y_test_preds)

    rslts.append(
        {
            "region": region,
            "model": model,
            "logs": logs,
            "roc": (fpr, tpr, thresholds),
            "auc": auc(fpr, tpr),
            "X_test_tensor": X_test_tensor,
            "y_test_tensor": y_test_tensor,
        }
    )


In [ ]:
plot_rslts_grids(rslts)

### Wider sidebands

Another approach is to widen the side-bands. The logic behind this is that, if the bump in the $m_{jj}$ distribution is around the same size as the mass window, the model might not be able to properly learn what is background and what is signal. Thus, by extending the side-bands, we allow more events that are outside of this potentially wide bump.

In [ ]:
# Parameters
bkgregion_width = 200
sigregion_width = 200
m0_interval = 100
m0_min = 2850
m0_max = 5000
m0s = np.arange(m0_min, m0_max + 1, m0_interval)

regions = make_regions(m0s, sigregion_width, bkgregion_width)

# Plot all m0 as vertical lines
for region in regions:
    plt.axvline(region["m0"], color='gray', linestyle='--', alpha=0.5)
    plt.axvline(region["sigregion"][1], color='red', linestyle='-', alpha=0.7)
    plt.axvline(region["sigregion"][0], color='blue', linestyle='-', alpha=0.7)
    plt.text(region["m0"], 50, f"{region['m0']}", rotation=90, verticalalignment='bottom', horizontalalignment='right', fontsize=8)
    plt.axvspan(region["bkgregion_left"][0], region["bkgregion_left"][1], color='orange', alpha=0.3)
    plt.axvspan(region["sigregion"][0], region["sigregion"][1], color='blue', alpha=0.3)
    plt.axvspan(region["bkgregion_right"][0], region["bkgregion_right"][1], color='orange', alpha=0.3)
plt.hist(df["m_jj"], bins=100, range=(2500, 6800))
plt.xlabel(r"$m_{jj}$")
plt.ylabel("Counts")
plt.title("Dijet Mass with Signal and Background Regions")
plt.show()

print("Mass hypotheses (m0s) used (GeV):")
print(m0s)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
rslts = []

for region in regions:
    print(region)
    set_deterministic(seed=SEED)

    train_loader, val_loader, test_loader, dfs, dss = dataset_generator(df, input_vars, region, batch_size=BATCH_SIZE)

    model = MLP(input_dim=len(input_vars), hidden_dim=HIDDEN_DIM, hidden_layers=4)

    if len(dfs["train"]) / model.count_trainable_params() <= 10:
        print("Not enough training data for the number of parameters in the model.")
        print(f"Train size: {len(dfs['train'])}, Val size: {len(dfs['val'])}, Test size: {len(dfs['test'])}")
        break

    model, logs = train(model, n_epochs=N_EPOCHS, lr=LR, train_loader=train_loader, val_loader=val_loader)
    
    with torch.no_grad():
        X_test_tensor, y_test_tensor = dss["test"].tensors
        y_test_preds = model(X_test_tensor.to(device)).cpu().numpy()
    fpr, tpr, thresholds = roc_curve(y_test_tensor, y_test_preds)

    rslts.append(
        {
            "region": region,
            "model": model,
            "logs": logs,
            "roc": (fpr, tpr, thresholds),
            "auc": auc(fpr, tpr),
            "X_test_tensor": X_test_tensor,
            "y_test_tensor": y_test_tensor,
        }
    )


In [ ]:
plot_rslts_grids(rslts)

## Wider signal region

This approach follows a similar logic as the previous experiment where we widened the side-bands. By widening the signal region, more of the bump will be labeled signal and the model might be able to learn to differentiate signal from background.

In [ ]:
# Parameters
bkgregion_width = 100
sigregion_width = 300
m0_interval = 100
m0_min = 2850
m0_max = 5000
m0s = np.arange(m0_min, m0_max + 1, m0_interval)

regions = make_regions(m0s, sigregion_width, bkgregion_width)

# Plot all m0 as vertical lines
for region in regions:
    plt.axvline(region["m0"], color='gray', linestyle='--', alpha=0.5)
    plt.axvline(region["sigregion"][1], color='red', linestyle='-', alpha=0.7)
    plt.axvline(region["sigregion"][0], color='blue', linestyle='-', alpha=0.7)
    plt.text(region["m0"], 50, f"{region['m0']}", rotation=90, verticalalignment='bottom', horizontalalignment='right', fontsize=8)
    plt.axvspan(region["bkgregion_left"][0], region["bkgregion_left"][1], color='orange', alpha=0.3)
    plt.axvspan(region["sigregion"][0], region["sigregion"][1], color='blue', alpha=0.3)
    plt.axvspan(region["bkgregion_right"][0], region["bkgregion_right"][1], color='orange', alpha=0.3)
plt.hist(df["m_jj"], bins=100, range=(2500, 6800))
plt.xlabel(r"$m_{jj}$")
plt.ylabel("Counts")
plt.title("Dijet Mass with Signal and Background Regions")
plt.show()

print("Mass hypotheses (m0s) used (GeV):")
print(m0s)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
rslts = []

for region in regions:
    print(region)
    set_deterministic(seed=SEED)

    train_loader, val_loader, test_loader, dfs, dss = dataset_generator(df, input_vars, region, batch_size=BATCH_SIZE)

    model = MLP(input_dim=len(input_vars), hidden_dim=HIDDEN_DIM, hidden_layers=4)

    if len(dfs["train"]) / model.count_trainable_params() <= 10:
        print("Not enough training data for the number of parameters in the model.")
        print(f"Train size: {len(dfs['train'])}, Val size: {len(dfs['val'])}, Test size: {len(dfs['test'])}")
        break

    model, logs = train(model, n_epochs=N_EPOCHS, lr=LR, train_loader=train_loader, val_loader=val_loader)
    
    with torch.no_grad():
        X_test_tensor, y_test_tensor = dss["test"].tensors
        y_test_preds = model(X_test_tensor.to(device)).cpu().numpy()
    fpr, tpr, thresholds = roc_curve(y_test_tensor, y_test_preds)

    rslts.append(
        {
            "region": region,
            "model": model,
            "logs": logs,
            "roc": (fpr, tpr, thresholds),
            "auc": auc(fpr, tpr),
            "X_test_tensor": X_test_tensor,
            "y_test_tensor": y_test_tensor,
        }
    )


In [ ]:
plot_rslts_grids(rslts)

For this particular approach, the training of the model for $m_{jj} = 3450 \text{GeV}$ does too much over training as can be seen for the other mass hypotheses. In fact, motivated by this observation, if one compares the AUC vs. $m_0$ plot between all of the difference approaches taken, one can see that models trained on and around this mass hypothesis tend yield a slightly higher AUC. While this is not conclusive evidence that there is something there, it would be something worth investigating further in future work.

## Conclusion

These results show no sign of detection of the bump in the di-jet mass distribution. The work done so far has consisted of a simplified CWoLa approach, which means that the obvious next steps in this search would be a more careful and rigurous implementation of this technique. This could involve a stronger decorellation method, a more careful binning strategy for the mass window generation, the implementation of a cross-validation strategy, etc. These approaches have the potential to increase our sensitivity to the resonance that we were searching for, providing us to have a better chance of finding the mass bump, and to investigate potential signs of it such as the one observed for $m_{jj} = 3450 \text{ GeV}$.

## EXTRA: Signal detection? (Probably not...)

The following preliminary results were obtained in the process of developing this notebook and is kept here as it serves a the only clear sign of a potential signal. Unfortunately, it's highly likely that this result is a false positive, as the initial implementation of the `make_regions` function had an issue where it would make regions too wide, which might have resulted in the window spilling over into the peak of the $m_jj$ distribution. This is an issue since one of the requirements of the applied technique is that the distribution be smooth. 

In [ ]:
# Parameters
bkgregion_width = 100
sigregion_width = 200
m0_min = 2800
m0_max = 3200
m0_interval = 150
m0s = np.arange(m0_min, m0_max + 1, m0_interval)

regions = make_regions(m0s, sigregion_width, bkgregion_width)

# Plot all m0 as vertical lines
for region in regions:
    plt.axvline(region["m0"], color='gray', linestyle='--', alpha=0.5)
    # Red right line for signal region
    plt.axvline(region["sigregion"][1], color='red', linestyle='-', alpha=0.7)
    # blue left line for signal region
    plt.axvline(region["sigregion"][0], color='blue', linestyle='-', alpha=0.7)
    plt.axvline(region["bkgregion_left"][0], color='green', linestyle='-', alpha=0.7)
    plt.axvline(region["bkgregion_right"][1], color='green', linestyle='-', alpha=0.7)
    plt.text(region["m0"], 50, f"{region['m0']}", rotation=90, verticalalignment='bottom', horizontalalignment='right', fontsize=8)
plt.hist(df["m_jj"], bins=100, range=(m0_min - sigregion_width, m0_max + sigregion_width))
plt.xlabel(r"$m_{jj}$")
plt.ylabel("Counts")
plt.title("Dijet Mass with Signal and Background Regions")
plt.show()

In [ ]:
regions[0]

In [ ]:
# Trainining hyperparameters
N_EPOCHS = 25
HIDDEN_DIM = 16
HIDDEN_LAYERS = 3
BATCH_SIZE = 256
LR = 0.01

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
rslts_2 = []

for region in regions:
    print(region)

    train_loader, val_loader, test_loader, dfs, dss = dataset_generator(df, input_vars, region, batch_size=BATCH_SIZE)

    model = MLP(input_dim=len(input_vars), hidden_dim=HIDDEN_DIM, hidden_layers=HIDDEN_LAYERS)

    if len(dfs["train"]) / model.count_trainable_params() <= 10:
        print("Not enough training data for the number of parameters in the model.")
        print(f"Train size: {len(dfs['train'])}, Val size: {len(dfs['val'])}, Test size: {len(dfs['test'])}")
        break

    model, logs = train(model, n_epochs=N_EPOCHS, lr=LR, train_loader=train_loader, val_loader=val_loader)
    
    with torch.no_grad():
        X_test_tensor, y_test_tensor = dss["test"].tensors
        y_test_preds = model(X_test_tensor.to(device)).cpu().numpy()
    fpr, tpr, thresholds = roc_curve(y_test_tensor, y_test_preds)

    rslts_2.append(
        {
            "region": region,
            "model": model,
            "logs": logs,
            "roc": (fpr, tpr, thresholds),
            "auc": auc(fpr, tpr),
            "X_test_tensor": X_test_tensor,
            "y_test_tensor": y_test_tensor,
            "dfs": dfs,
            "dss": dss
        }
    )

In [ ]:
# Make grid of roc curves from rslts. With 3 columns
n_cols = 3
n_rows = int(np.ceil(len(rslts_2) / n_cols))
fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 5 * n_rows))
axes = axes.flatten()
for i, rslt in enumerate(rslts_2):
    fpr, tpr, thresholds = rslt["roc"]
    ax = axes[i]
    ax.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % rslt["auc"])
    ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title(f"ROC for $m_0$ = {rslt['region']['m0']} GeV")
    ax.legend(loc="lower right")
    ax.grid()
plt.show()

# Make grid of loss plots from rslts. With 3 columns
n_cols = 3
n_rows = int(np.ceil(len(rslts_2) / n_cols))
fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 5 * n_rows))
axes = axes.flatten()
for i, rslt in enumerate(rslts_2):
    ax = axes[i]
    ax.plot(rslt["logs"]["loss"], label="Training Loss")
    ax.plot(rslt["logs"]["val_loss"], label="Validation Loss")
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Loss")
    ax.legend()
    ax.set_title(f"Training and Validation Loss ($m_0$ = {rslt['region']['m0']} GeV)")
    ax.grid()
plt.show()

In [ ]:
# Plot AUC vs mass hypothesis
m0s = [rslt["region"]["m0"] for rslt in rslts_2]
aucs = [rslt["auc"] for rslt in rslts_2]
plt.plot(m0s, aucs, marker='o')
plt.xlabel(r"$m_0$ [GeV]")
plt.ylabel("AUC")
plt.title("AUC vs Mass Hypothesis")
plt.grid()
plt.show()

In [ ]:
THRESHOLD=0.6

with torch.no_grad():
    test_scores = rslts_2[0]["model"](rslts_2[0]["X_test_tensor"].to(device)).cpu().numpy()
plt.hist(test_scores, bins=100, range=(0,1))
plt.xlabel("Model Output Score")
plt.ylabel("Counts")
plt.title("Model Output Scores on Test Set")

In [ ]:
# Make dijet mass histogram for events passing threshold along with all events dijet mass
df_test_pass = rslts_2[0]["dfs"]["test"][test_scores > THRESHOLD]
plt.hist(df_test_pass["m_jj"], bins=100, range=(2500, 6500), alpha=0.5, label="Passed", density=True)
plt.hist(rslts_2[0]["dfs"]["test"]["m_jj"], bins=100, range=(2500, 6500), alpha=0.5, label="All", density=True)
plt.xlabel(r"$m_{jj}$")

In [ ]:
# Applying model to full dataset for one of the mass points
test_input_data = dataset_generator(df, input_vars, regions[1], batch_size=BATCH_SIZE)
with torch.no_grad():
    preds = rslts_2[1]["model"](
        test_input_data[4]["test"].tensors[0].to(device)
    ).cpu().numpy()
preds

In [ ]:
test_df = test_input_data[3]["test"]
test_df["model_score"] = preds
# Plot model score distribution
plt.hist(test_df["model_score"], bins=100, range=(0,1))
plt.xlabel("Model Output Score")
plt.ylabel("Counts")
plt.title("Model Output Scores on Test Set")
plt.show()